In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
data_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [19]:
trainset = datasets.CIFAR100(root='./data', train = True, transform = data_transforms, download=True)

testset = datasets.CIFAR100(root='./data', train=False, transform = data_transforms,
download=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=256,shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=256,shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [20]:
model = models.vgg16(pretrained=True)

num_in_ftrs = model.classifier[6].in_features
num_cls = len(trainset.classes)
model.classifier[6] = nn.Linear(num_in_ftrs, num_cls)

for param in model.parameters(): # freeze all the layers
  param.requires_grad = False

for param in model.classifier[6].parameters(): # unfreeze the last linear layer.
  param.requires_grad = True

In [21]:
num_epochs = 30
# num_epochs = 15
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [22]:
train = int(input("Enter 1 for training : "))
if train == 1:
  model.train()
  for epoch in range(num_epochs):

      loss = 0.0
      for i, data in enumerate(trainloader, 0):
          images, labels = data
          # images, labels = images.cuda(), labels.cuda() # For GPU use
          
          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          loss += loss.item()
          if i % 2000 == 1999:    # print every mini-batches
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, loss / 2000))
              loss = 0.0

  print('Finished Training')
  torch.save(model.state_dict(), 'best_model.pth')
  # torch.save(best_model_wts , '/content/drive/MyDrive/computer_vision/PR2/part1/best_model_weight_PR2_13.pth')
else:
  # model.load_state_dict(torch.load('best_model.pth'))
  model.load_state_dict(torch.load('/content/drive/MyDrive/computer_vision/PR2/part1/best_model_weight_PR2_2.pth', map_location=torch.device('cpu')))
  model.eval()

Enter 1 for training : 0


In [ ]:
true_pos = 0
total_detect = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        # images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total_detect += labels.size(0)
        true_pos += (preds == labels).sum().item()

print('Accuracy : %d %%' % (
    100 * true_pos / total_detect))

Batch size - 16 -  
epoch  = 10
accuracy - 54

---

Batch size - 64 -  
epoch  = 10
accuracy - 61

---

Batch size - 64 -  
learning_rate = 0.0001
epoch  = 10
accuracy - 49

---

Batch size - 128 -  
learning_rate = 0.001
epoch  = 15
accuracy - 56

---
Batch size - 256 -  
learning_rate = 0.0001
epoch  = 50
accuracy - 52 - Failed - Ask to use pro for long time
Started at 3.09 am - check the file creation time to see the trainning ending

---
Batch size - 256 -  
learning_rate = 0.0001
epoch  = 30
accuracy - 55
Started at 10.44 am - check the file creation time to see the trainning ending


